# Tarea 1
Se cargan los datos correspondientes a la tarea 1, la matriz de pesos extraídos de fasttext (embedding layer) y el diccionario de palabras: int. Las id de las palabras en ese diccionario representan el índice en donde se encuentra el vector de esa palabra en el embedding layer.

Los identificadores de labels (conceptos) son convertidos a one hot vector

La función preprocess_x tiene como objetivo encontrar la secuencia de palabras (argumento) más largo, de tal forma de luego hacer padding a las entradas más pequeñas. Para no tener errores probando otras entradas más largas, se aumentó este valor en 100 para tener rango de error.

In [1]:
import pandas as pd
import numpy as np
import logging
from itertools import product
import unicodedata
import re
import pickle

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics

from keras.utils import np_utils
from keras import regularizers
from keras.models import Sequential
from keras.optimizers import SGD
from keras import regularizers
from keras.models import load_model, Model
from keras.layers import Dense, Dropout, Input, Embedding, Lambda
from keras import backend as K
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import text_to_word_sequence
from keras import optimizers

Using TensorFlow backend.


## Load data

In [2]:
df = []
for tema in range(0, 4):
    dict_column = {}
    for column in ['x', 'y']:
        dict_set = {}
        for set_ in ['train', 'dev', 'test']:
            filename = '../../data/'+column+'_'+set_+'_tema_'+str(tema+1)+'_categorias_pnud_0.txt'
            with open(filename) as f:
                data = f.readlines()
            dict_set[set_] = [row[:-1] for row in data]
        dict_column[column] = dict_set
    df.append(dict_column)

In [3]:
print(df[0].keys())
print(df[0]['x'].keys())

dict_keys(['x', 'y'])
dict_keys(['train', 'dev', 'test'])


### Vectors and Embedding layer

In [4]:
from gensim.models.wrappers import FastText
model = FastText.load_word2vec_format('../../word_vectors/ca-vectors.vec')

In [5]:
EMBEDDING_DIM = 300
vocab_size = len(model.vocab)
embedding_matrix = np.zeros((vocab_size+1, EMBEDDING_DIM))
for word in model.vocab:
    index = model.vocab[word].index
    embedding_matrix[index+1, :] = model[word]

## Prepare data

In [6]:
global encoder       # to detect number of classes

In [7]:
def get_y_sets(NUM_DF):
    TRAIN_SIZE = len(df[NUM_DF]['x']['train'])
    DEV_SIZE = len(df[NUM_DF]['x']['dev'])
    TEST_SIZE = len(df[NUM_DF]['x']['test'])
    df_y = np.array(df[NUM_DF]['y']['train'] + df[NUM_DF]['y']['dev'] + df[NUM_DF]['y']['test'])
    # one hot vector label for clasification
    global encoder
    encoder = LabelEncoder()
    encoder.fit(df_y) # to know how many classes 
    labels = encoder.transform(df_y)
    Y = to_categorical(np.asarray(labels))
    
    y_train = Y[0 : TRAIN_SIZE]
    y_dev = Y[TRAIN_SIZE : TRAIN_SIZE+DEV_SIZE]
    y_test = Y[TRAIN_SIZE+DEV_SIZE : ]
    return y_train, y_dev, y_test

In [8]:
global df_x, sequences, MAX_SEQUENCE_LENGTH
df_x = [None, None, None, None]              # argumentos de train+dev+test para cada tema
sequences = [None, None, None, None]         # argumentos como listas de palabras para cada tema
MAX_SEQUENCE_LENGTH = 0                      # tamaño máximo de vector de entrada

In [28]:
def preprocess_x(num_df):
    '''
    Converts arguments in word sequence saving it in global sequences array 
    Updates the global max sequence length.
    
    Arguments:
        num_df: theme number to process
    '''
    global MAX_SEQUENCE_LENGTH, df_x, sequences
    df_x[num_df] = np.array(df[num_df]['x']['train'] + df[num_df]['x']['dev'] + df[num_df]['x']['test'])
    # to list of words
    sequences[num_df] = []
    for argument_j in range(0, df_x[num_df].shape[0]):
        in_string = df_x[num_df][argument_j]
        sequences[num_df].append(text_to_word_sequence(in_string))
    # search for the biggest
    for sequence in sequences[num_df]:
        if len(sequence) > MAX_SEQUENCE_LENGTH:
            MAX_SEQUENCE_LENGTH = len(sequence)

def get_x_sets(num_df):
    '''
    Replaces word in sequences for corresponding numbers.
    Arguments:
        num_df: theme number from which to get the sets
    Returns:
        Train, development and test set
    '''
    global df_x, sequences
    # every X[i] with max size
    # replace words by numbers with world_dict
    index2word_set = set(model.index2word)
    X = np.zeros((df_x[num_df].shape[0], MAX_SEQUENCE_LENGTH)).astype(int)
    for i in range(0, len(sequences[num_df])):
        for j in range(0, len(sequences[num_df][i])):
            word = sequences[num_df][i][j]
            if word in index2word_set: 
                X[i][-len(sequences[num_df][i])+j] = model.vocab[word].index+1
            else:
                X[i][-len(sequences[num_df][i])+j] = 0
    # divide sets for answer
    TRAIN_SIZE = len(df[num_df]['x']['train'])
    DEV_SIZE = len(df[num_df]['x']['dev'])
    TEST_SIZE = len(df[num_df]['x']['test'])
    X_train = X[0 : TRAIN_SIZE]
    X_dev = X[TRAIN_SIZE : TRAIN_SIZE+DEV_SIZE]
    X_test = X[TRAIN_SIZE+DEV_SIZE : ]
    return X_train, X_dev, X_test 

In [29]:
for i in range(0, 4):
    preprocess_x(i)

In [30]:
sequences[3][2]

['poder',
 'judicial',
 'probo',
 'ecuánime',
 'y',
 'equitativo',
 'respeto',
 'absoluto',
 'a',
 'la',
 'ley']

In [31]:
MAX_SEQUENCE_LENGTH

500

In [32]:
MAX_SEQUENCE_LENGTH = 500

## Train best configuration models

In [33]:
def get_rec_prec_f1(scores):
    # transforms scores to one hot classification
    index_max = np.argmax(scores, axis=1)
    y_pred = np.zeros(scores.shape)
    for i in range(0, len(index_max)):
        y_pred[i][index_max[i]]=1
    rec = round(100*metrics.recall_score(y_test, y_pred, average='weighted'), 1)
    prec = round(100*metrics.precision_score(y_test, y_pred, average='weighted'), 1)
    f1 = round(100*metrics.f1_score(y_test, y_pred, average='weighted'), 1)
    return rec, prec, f1

In [43]:
from keras.callbacks import ModelCheckpoint

def dan(relu_layers=3, hidden_units=300, p_dropout=0.3, dropout_input=False, 
        my_regularizer=regularizers.l2(1e-5), my_optimizer='adam', 
        epochs=150, batch_size=200, trainable=False, filepath=''):
    '''
    Creates and fit NN. 
    NN Arquitecture: 
        Input: vector with numbers representing index in embedding layer
        Embedding layer: matrix multiplication to obtain vectors for each index in the input
        Dropout: optional words dropout
        Mean: Averages the vectors of the embedding output, returning one averaged vector
        Fully connected: Fully connected layers with relu as activation function
                         optional neuron dropout
        Fully connected: output layer with softmax function
    
    Arguments:
        relu_layers: Number of fully connected layers with relu 
        hidden_units: Number of neurons on the relu layers
        p_dropout: dropout probability for the relu layers
        dropout_input: dropout probability for words 
        my_regularizer: kernel_regularizer for fully connected layers
        my_optimizer: optimizer for back propagation
        epochs: number of epochs to train
        batch_size: batch_size for trainning
    '''
    sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int64')
    x = Embedding(len(model.vocab)+1, EMBEDDING_DIM, mask_zero=True,
                  weights=[embedding_matrix], trainable=trainable)(sequence_input)
    if dropout_input:
        x = Dropout(0.2)(x)
    x = Lambda(lambda x: K.mean(x, axis=1), 
               output_shape=(embedding_matrix.shape[1],))(x)
    for i in range(0, relu_layers):
        x = Dropout(p_dropout)(x)
        x = Dense(units=hidden_units, activation='relu', kernel_regularizer=my_regularizer)(x)
    preds = Dense(units=len(encoder.classes_), activation='softmax', 
                  kernel_regularizer=my_regularizer)(x)
    
    m = Model(sequence_input, preds)
    m.compile(loss='categorical_crossentropy', optimizer=my_optimizer, 
              metrics=['accuracy', 'top_k_categorical_accuracy'])
    print ("Starting trainning")
    print(relu_layers, hidden_units, p_dropout, dropout_input, 
          my_regularizer, my_optimizer, epochs, batch_size, 
          trainable, filepath)
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, 
                                 save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    
    m.fit(X_train, y_train,
          validation_data=(X_dev, y_dev),
          shuffle=True, 
          epochs=epochs, batch_size=batch_size,
          verbose=0,
          callbacks=callbacks_list)
    return m

In [45]:
relu_layers=2
hidden_units=200
p_dropout=0.2
batch_size = 30
epochs = 100

for i in range(0, 4):
    print("------------- TEMA "+str(i+1)+" --------------")
    y_train, y_dev, y_test = get_y_sets(i)
    X_train, X_dev, X_test = get_x_sets(i)
    l2 = None if i != 1 else regularizers.l2(1e-5) # the best has not regularizator in theme 2
    m = dan(relu_layers=relu_layers, hidden_units=hidden_units, 
            p_dropout=p_dropout, my_regularizer=l2,
            epochs=epochs, batch_size=batch_size,
            filepath='models/checkpoints/best_tema_' + str(i+1) + '.h5')
    m = load_model('models/checkpoints/best_tema_' + str(i+1) + '.h5')
    results = m.evaluate(X_test, y_test)
    print("%s: %0.2f" % (m.metrics_names[1], round(100*results[1], 2))) # accuracy
    print("%s: %0.2f" % (m.metrics_names[2], round(100*results[2], 2))) # top-5
    scores = m.predict(X_test, batch_size=batch_size)
    rec, prec, f1 = get_rec_prec_f1(scores)
    print("Recall: ", rec , "\nPrecision: ", prec, "\nF1: ", f1)

------------- TEMA 1 --------------
Starting trainning
2 200 0.2 False None adam 100 30 False models/checkpoints/best_tema_1.h5
Epoch 00000: val_acc improved from -inf to 0.37129, saving model to models/checkpoints/best_tema_1.h5
Epoch 00001: val_acc improved from 0.37129 to 0.44416, saving model to models/checkpoints/best_tema_1.h5
Epoch 00002: val_acc improved from 0.44416 to 0.50293, saving model to models/checkpoints/best_tema_1.h5
Epoch 00003: val_acc improved from 0.50293 to 0.53437, saving model to models/checkpoints/best_tema_1.h5
Epoch 00004: val_acc improved from 0.53437 to 0.54999, saving model to models/checkpoints/best_tema_1.h5
Epoch 00005: val_acc improved from 0.54999 to 0.56951, saving model to models/checkpoints/best_tema_1.h5
Epoch 00006: val_acc improved from 0.56951 to 0.57601, saving model to models/checkpoints/best_tema_1.h5
Epoch 00007: val_acc improved from 0.57601 to 0.58599, saving model to models/checkpoints/best_tema_1.h5
Epoch 00008: val_acc did not improv

/Users/constanzafierro/tensorflow-p3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/constanzafierro/tensorflow-p3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Recall:  64.4 
Precision:  65.2 
F1:  63.9
------------- TEMA 2 --------------
Starting trainning
2 200 0.2 False <keras.regularizers.L1L2 object at 0x15fdb6fd0> adam 100 30 False models/checkpoints/best_tema_2.h5
Epoch 00000: val_acc improved from -inf to 0.35255, saving model to models/checkpoints/best_tema_2.h5
Epoch 00001: val_acc improved from 0.35255 to 0.47426, saving model to models/checkpoints/best_tema_2.h5
Epoch 00002: val_acc improved from 0.47426 to 0.53809, saving model to models/checkpoints/best_tema_2.h5
Epoch 00003: val_acc improved from 0.53809 to 0.56577, saving model to models/checkpoints/best_tema_2.h5
Epoch 00004: val_acc improved from 0.56577 to 0.59208, saving model to models/checkpoints/best_tema_2.h5
Epoch 00005: val_acc improved from 0.59208 to 0.59780, saving model to models/checkpoints/best_tema_2.h5
Epoch 00006: val_acc improved from 0.59780 to 0.61084, saving model to models/checkpoints/best_tema_2.h5
Epoch 00007: val_acc improved from 0.61084 to 0.61794,

Epoch 00025: val_acc improved from 0.73262 to 0.73386, saving model to models/checkpoints/best_tema_3.h5
Epoch 00026: val_acc did not improve
Epoch 00027: val_acc did not improve
Epoch 00028: val_acc improved from 0.73386 to 0.73586, saving model to models/checkpoints/best_tema_3.h5
Epoch 00029: val_acc did not improve
Epoch 00030: val_acc did not improve
Epoch 00031: val_acc did not improve
Epoch 00032: val_acc did not improve
Epoch 00033: val_acc did not improve
Epoch 00034: val_acc did not improve
Epoch 00035: val_acc did not improve
Epoch 00036: val_acc did not improve
Epoch 00037: val_acc did not improve
Epoch 00038: val_acc did not improve
Epoch 00039: val_acc did not improve
Epoch 00040: val_acc improved from 0.73586 to 0.73686, saving model to models/checkpoints/best_tema_3.h5
Epoch 00041: val_acc improved from 0.73686 to 0.73760, saving model to models/checkpoints/best_tema_3.h5
Epoch 00042: val_acc did not improve
Epoch 00043: val_acc improved from 0.73760 to 0.73785, saving 

Epoch 00068: val_acc did not improve
Epoch 00069: val_acc did not improve
Epoch 00070: val_acc did not improve
Epoch 00071: val_acc did not improve
Epoch 00072: val_acc did not improve
Epoch 00073: val_acc did not improve
Epoch 00074: val_acc did not improve
Epoch 00075: val_acc did not improve
Epoch 00076: val_acc improved from 0.68677 to 0.68728, saving model to models/checkpoints/best_tema_4.h5
Epoch 00077: val_acc did not improve
Epoch 00078: val_acc did not improve
Epoch 00079: val_acc did not improve
Epoch 00080: val_acc did not improve
Epoch 00081: val_acc improved from 0.68728 to 0.68753, saving model to models/checkpoints/best_tema_4.h5
Epoch 00082: val_acc did not improve
Epoch 00083: val_acc did not improve
Epoch 00084: val_acc did not improve
Epoch 00085: val_acc did not improve
Epoch 00086: val_acc did not improve
Epoch 00087: val_acc improved from 0.68753 to 0.68931, saving model to models/checkpoints/best_tema_4.h5
Epoch 00088: val_acc did not improve
Epoch 00089: val_ac

In [46]:
for i in range(0, 4):
    print("------------- TEMA "+str(i+1)+" --------------")
    y_train, y_dev, y_test = get_y_sets(i)
    X_train, X_dev, X_test = get_x_sets(i)
    
    m = load_model('models/checkpoints/best_tema_' + str(i+1) + '.h5')
    results = m.evaluate(X_test, y_test)
    print("%s: %0.2f" % (m.metrics_names[1], round(100*results[1], 2))) # accuracy
    print("%s: %0.2f" % (m.metrics_names[2], round(100*results[2], 2))) # top-5
    scores = m.predict(X_test, batch_size=batch_size)
    rec, prec, f1 = get_rec_prec_f1(scores)
    print("Recall: ", rec , "\nPrecision: ", prec, "\nF1: ", f1)

------------- TEMA 1 --------------
4480/4611 [============================>.] - ETA: 0sacc: 64.43
top_k_categorical_accuracy: 89.46


/Users/constanzafierro/tensorflow-p3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/constanzafierro/tensorflow-p3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Recall:  64.4 
Precision:  65.2 
F1:  63.9
------------- TEMA 2 --------------
4320/4370 [============================>.] - ETA: 0sacc: 67.78
top_k_categorical_accuracy: 91.12
Recall:  67.8 
Precision:  67.9 
F1:  66.7
------------- TEMA 3 --------------
3968/4013 [============================>.] - ETA: 0sacc: 74.16
top_k_categorical_accuracy: 96.11
Recall:  74.2 
Precision:  74.2 
F1:  73.9
------------- TEMA 4 --------------
3840/3930 [============================>.] - ETA: 0sacc: 66.46
top_k_categorical_accuracy: 91.40
Recall:  66.5 
Precision:  66.7 
F1:  65.8
